#Skin Disease Image Classification Notebook

In this notebook, we're making an image classifier which can classify six skin diseases: 
* Acne and Rosacea
* Atopic Dermatitis
* Herpes HPV and other STD
* Poison Ivy and other Contact Diseases
* Psoriasis or Lichen
* Scabies Lyme Disease and other Infestations and Bites
 
The full dataaset can be found here https://www.kaggle.com/datasets/shubhamgoel27/dermnet

Sometimes these skin deseases look similar and indistinguishable with the regular symptoms such as rashes and redish. 

Can we use deep learning to classify these deseases? What model architecture should we use? How should we train our models? This project hopes to answer these questions/

#Import Library

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import matplotlib.image as mpimg
%matplotlib inline
import random
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from google.cloud import storage

# Download DATASET
You can choose between this two methods below 

## Google Drive
Google Drive is faster to use

In [ ]:
# TO RUN THIS LOCALLY MAKE SURE YOU INSTALL THE MODULE FIRST
import gdown
!gdown --folder https://drive.google.com/drive/folders/1uRPA1LJPmc6QHp-IGVdpcMYz52daxBxF?usp=sharing

Retrieving folder list
Processing file 1fumMrJhoBGfqZtrcnBOAvngpjxND49Hp image_dataset.zip
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1fumMrJhoBGfqZtrcnBOAvngpjxND49Hp
To: /content/Dataset/image_dataset.zip
100% 462M/462M [00:06<00:00, 73.7MB/s]
Download completed


In [ ]:
#Mounting BANGKIT GDrive

# Change the PATH with ur own DATASET location
# Shared Gdrive for Dataset https://drive.google.com/drive/folders/1oYBG6KJSZjtlSF3wy7eb-KZF66J070Zc?usp=sharing
# from google.colab import drive
# drive.mount('/content/drive')

## Unzipping The file

In [ ]:
import shutil
# Change the PATH for with your own Directory
filename = '/content/Dataset/image_dataset.zip'
shutil.unpack_archive(filename)

## Google Cloud

In [ ]:
# TO RUN THIS LOCALLY MAKE SURE YOU INSTALL THE MODULE FIRST
# Downloading TRAIN dataset
import gsutil
!gsutil cp -r gs://c22ps191b2/dataset/Train/ .

In [ ]:
# Download TEST dataset
!gsutil cp -r gs://c22ps191b2/dataset/Test/ .

# TEST DISPLAY DATASET

In [ ]:
#Test Display
# Change the PATH with ur own DATASET location
from IPython.display import Image
Image("/content/drive/MyDrive/Datasets/Train/Acne and Rosacea Photos/07Acne081101.jpg")

In [ ]:
# Test Display (2)
# Change the PATH with ur own DATASET location
plt.figure(figsize=(20,20))
test_folder=r'/content/drive/MyDrive/Datasets/Test/Acne and Rosacea Photos'
for i in range(5):
    file = random.choice(os.listdir(test_folder))
    image_path= os.path.join(test_folder, file)
    img=mpimg.imread(image_path)
    ax=plt.subplot(1,5,i+1)
    ax.title.set_text(file)
    plt.imshow(img)

In [ ]:
# Change the PATH with ur own DATASET location
base_dir = '/content/drive/MyDrive/Datasets'

print("Contents of base directory:")
print(os.listdir(base_dir))

print("\nContents of train directory:")
print(os.listdir(f'{base_dir}/Train'))

print("\nContents of validation directory:")
print(os.listdir(f'{base_dir}/Test'))

Contents of base directory:
['Test', 'Train', 'class_list.txt']

Contents of train directory:
['Poison Ivy Photos and other Contact Dermatitis', 'Herpes HPV and other STDs Photos', 'Acne and Rosacea Photos', 'Atopic Dermatitis Photos', 'Scabies Lyme Disease and other Infestations and Bites', 'Psoriasis pictures Lichen Planus and related diseases']

Contents of validation directory:
['Poison Ivy Photos and other Contact Dermatitis', 'Psoriasis pictures Lichen Planus and related diseases', 'Acne and Rosacea Photos', 'Atopic Dermatitis Photos', 'Scabies Lyme Disease and other Infestations and Bites', 'Herpes HPV and other STDs Photos']


# Data Pre-Processing

In [ ]:
# Change the PATH with ur own DATASET location
train_path="/content/drive/MyDrive/Datasets/Train"
test_path="/content/drive/MyDrive/Datasets/Test"

In [ ]:
x_train=[]

for folder in os.listdir(train_path):

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(150,150))

        x_train.append(img_arr)

x_test=[]

for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(150,150))

        x_test.append(img_arr)

In [ ]:
train_x=np.array(x_train)
test_x=np.array(x_test)

In [ ]:
train_x=train_x/255.0
test_x=test_x/255.0

In [ ]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (150, 150),
                                                 batch_size = 10,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (150, 150),
                                            batch_size = 10,
                                            class_mode = 'sparse')

Found 3693 images belonging to 6 classes.
Found 1058 images belonging to 6 classes.


In [ ]:
train_y=training_set.classes
test_y=test_set.classes

In [ ]:
training_set.class_indices

{'Acne and Rosacea Photos': 0,
 'Atopic Dermatitis Photos': 1,
 'Herpes HPV and other STDs Photos': 2,
 'Poison Ivy Photos and other Contact Dermatitis': 3,
 'Psoriasis pictures Lichen Planus and related diseases': 4,
 'Scabies Lyme Disease and other Infestations and Bites': 5}

In [ ]:
train_y.shape,test_y.shape

((3693,), (1058,))

# MODELLING

In [ ]:
def create_model():      

  # Define the model
  model = tf.keras.models.Sequential([ 
    tf.keras.layers.Conv2D(128,(3,3), activation = 'relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(6, activation='softmax')
  ])
  

  model.compile(optimizer = RMSprop(learning_rate=0.001),
                loss =  'sparse_categorical_crossentropy',
                metrics=['accuracy'])
     
  
  return model

# COMPILING

In [ ]:
model = create_model()

In [ ]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [ ]:
history = model.fit(
  train_x,
  train_y,
  epochs=50,
  batch_size=32,shuffle=True)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import numpy as np
#predict
y_pred=model.predict(test_x)
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
#get classification report
print(classification_report(y_pred,test_y))

              precision    recall  f1-score   support

           0       0.20      0.29      0.24       214
           1       0.09      0.05      0.07       206
           2       0.02      0.01      0.02       144
           3       0.00      0.00      0.00        30
           4       0.31      0.27      0.29       406
           5       0.07      0.14      0.10        58

    accuracy                           0.18      1058
   macro avg       0.12      0.13      0.12      1058
weighted avg       0.18      0.18      0.18      1058



In [ ]:
#get confusion matrix
print(confusion_matrix(y_pred,test_y))

[[ 62  23  47   3  46  33]
 [ 13  11  51  54  73   4]
 [ 28   7   2   5  97   5]
 [ 10   2   1   0  15   2]
 [169  69   1   2 109  56]
 [ 26  11   0   1  12   8]]


# Model Testing

Referensi https://www.geeksforgeeks.org/python-image-classification-using-keras/

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = x / 255
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if max(classes[0])==classes[0][0]:
    print("Acne and Rosacea")
  elif max(classes[0])==classes[0][1]:
    print("Atopic Dermatitis")
  elif max(classes[0])==classes[0][2]:
    print("Herpes HPV and other STD")
  elif max(classes[0])==classes[0][3]:
    print("Poison Ivy and other contact diseases")
  elif max(classes[0])==classes[0][4]:
    print("Psoriasis or Lichen")
  elif max(classes[0])==classes[0][5]:
    print("Scabies Lyme Disease and other infestations and bites")  

Saving Lichen-Nitidus-3.jpg to Lichen-Nitidus-3.jpg
[2.3164639e-01 5.1888373e-06 2.7756953e-06 5.1630114e-04 7.6782840e-01
 8.7890504e-07]
Psoriasis


# Save the Model

To load the trained model into TensorFlow Serving we first need to save it in the [SavedModel](https://www.tensorflow.org/guide/saved_model) format.  This will create a protobuf file in a well-defined directory hierarchy, and will include a version number.  [TensorFlow Serving](https://www.tensorflow.org/tfx/serving/serving_config) allows us to select which version of a model, or "servable" we want to use when we make inference requests.  Each version will be exported to a different sub-directory under the given path.

In [ ]:
# JALANIN
# Save the trained model as a Keras HDF5 file. 
import time
saved_model_path = "./my_model.h5"#.format(int(time.time()))

model.save(saved_model_path)


##TF LITE CONVERT

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_litemodel = converter.convert()

with open('model.tflite','wb') as f:
  f.write(tf_litemodel)

INFO:tensorflow:Assets written to: /tmp/tmps98mhydh/assets


INFO:tensorflow:Assets written to: /tmp/tmps98mhydh/assets


# Converting the model to TFJS



In [ ]:
#!pip install tensorflow==2.2

In [ ]:
#!pip install tensorflowjs

In [ ]:
#print('\u2022 Using TensorFlow Version:', tf.__version__)

• Using TensorFlow Version: 2.8.2


In [ ]:
# Use the tensorflow.js converter to convert the saved Keras model into JSON format.

!tensorflowjs_converter --input_format=keras {saved_model_path} ./